In [ ]:
import geopandas as gpd
import pandas as pd
import math
import numpy as np

counties = gpd.read_file("/Users/zwy/Library/CloudStorage/Dropbox/work files/Policy impact on fire/county_cities_filtered(interaction).gpkg")
counties = counties.to_crs(epsg='4326')
counties.info()

In [ ]:
data_year = 2017
fire_df = pd.read_csv("/Users/zwy/Library/CloudStorage/Dropbox/work files/Policy impact on fire/VIIRS_{}_clipped.csv".format(data_year))
fire_df.info()
geometry =gpd.GeoSeries.from_wkt(fire_df["geometry"])
fire_points = gpd.GeoDataFrame(
    fire_df[["id", "date", "time", "brightness", "confidence"]], geometry=geometry
)

In [ ]:
# 如果使用inner merge，会忽略无数据的行
counties_with_fire=counties.merge(
		counties.sjoin(
			fire_points
		).groupby(
			'AD2004'
		)['AD2004'].count().rename(
			'fire'
		).reset_index(), on='AD2004', how='left'
	)

counties_with_fire.loc[:,"n_points"].replace(np.nan, 0, inplace=True)
counties_with_fire.info()

In [ ]:
# Save the result (.shp)
counties_with_fire.to_file("/Users/zwy/Library/CloudStorage/Dropbox/work files/Policy impact on fire/counties_with_fire_{}".format(data_year),encoding='gbk')